<style>
.custom-header {
    background-color: #eeeee4; 
    color: #eab676; 
    text-align: center; 
    border-radius: 12px;
    font-family: Italic, sans-serif;
    font-size: 30px; 
    box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);
}
</style>
<div class="custom-header">
    <i>Welcome to our Personality Classifier<br><i>
    Please make sure to read the "<b>README<b>" file to ensure the program runs successfully.

</div>


In [13]:
from IPython.display import HTML
display(HTML('<Center><img src="images/type_p.png"></center>'))

<style>
.custom-header {
    background-color: #900C3F; 
    color: white; 
    text-align: center; 
    border-radius: 12px;
    font-family: Italic, sans-serif;
    font-size: 40px; 
    box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);
}
</style>
<div class="custom-header">
    First Chapter: NLP Project Predicting Personality from Text
</div>


<h3><i>Import libraries for NLP model</i></h3>


In [14]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import numpy as np
import os

In [15]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ibrah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ibrah\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
if not os.path.exists('PreModel'):
    os.makedirs('PreModel')
df = pd.read_csv('MBTI 500.csv')
df = df.groupby('type').apply(lambda x: x.sample(n=2000, replace=True, random_state=42))
df = df.reset_index(drop=True)

In [17]:

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [18]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
class DataGenerator(Sequence):
    def __init__(self, X, y, batch_size, vectorizer):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.vectorizer = vectorizer

    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        batch_X = self.X[index * self.batch_size:(index + 1) * self.batch_size]
        batch_y = self.y[index * self.batch_size:(index + 1) * self.batch_size]
        X_vect = self.vectorizer.transform(batch_X).toarray()
        y_encoded = pd.factorize(batch_y)[0]
        return X_vect, y_encoded

In [19]:
df['posts'] = df['posts'].apply(clean_text).apply(lemmatize_text)


In [20]:
X = df['posts']
y = df['type']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
vectorizer = CountVectorizer(max_features=10000)
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

In [22]:
svc_model = LinearSVC()
svc_model.fit(X_train_counts, y_train)

LinearSVC()

In [23]:
svc_predictions = svc_model.predict(X_test_counts)
svc_accuracy = accuracy_score(y_test, svc_predictions)
print(f"SVC Model Accuracy: {svc_accuracy}")
print(classification_report(y_test, svc_predictions))

SVC Model Accuracy: 0.863125
              precision    recall  f1-score   support

        ENFJ       0.88      0.90      0.89       373
        ENFP       0.79      0.84      0.82       412
        ENTJ       0.87      0.88      0.88       409
        ENTP       0.76      0.72      0.74       401
        ESFJ       0.97      1.00      0.98       405
        ESFP       0.96      1.00      0.98       414
        ESTJ       0.97      1.00      0.98       414
        ESTP       0.96      0.95      0.96       446
        INFJ       0.75      0.75      0.75       395
        INFP       0.77      0.68      0.72       387
        INTJ       0.72      0.63      0.67       390
        INTP       0.69      0.67      0.68       371
        ISFJ       0.93      0.97      0.95       408
        ISFP       0.91      0.96      0.94       390
        ISTJ       0.90      0.92      0.91       392
        ISTP       0.90      0.89      0.89       393

    accuracy                           0.86      64

In [24]:
joblib.dump(svc_model, 'PreModel/svc_mbti_classifier.pkl')
joblib.dump(vectorizer, 'PreModel/count_vectorizer.pkl')

['PreModel/count_vectorizer.pkl']

In [25]:

keras_model = Sequential()
keras_model.add(Dense(128, activation='relu', input_shape=(X_train_counts.shape[1],)))
keras_model.add(Dense(64, activation='relu'))
keras_model.add(Dense(len(df['type'].unique()), activation='softmax'))

keras_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 32
train_generator = DataGenerator(X_train.tolist(), y_train.tolist(), batch_size, vectorizer)
test_generator = DataGenerator(X_test.tolist(), y_test.tolist(), batch_size, vectorizer)


In [26]:
keras_model.fit(train_generator, epochs=5, verbose=1, validation_data=test_generator)

Epoch 1/5
800/800 [==============================] - 25s 28ms/step - loss: 2.6694 - accuracy: 0.0864 - val_loss: 2.6668 - val_accuracy: 0.0808
Epoch 2/5
800/800 [==============================] - 22s 27ms/step - loss: 2.5840 - accuracy: 0.1361 - val_loss: 2.7180 - val_accuracy: 0.0817
Epoch 3/5
800/800 [==============================] - 23s 29ms/step - loss: 2.4159 - accuracy: 0.2096 - val_loss: 2.8445 - val_accuracy: 0.0794
Epoch 4/5
800/800 [==============================] - 23s 29ms/step - loss: 2.1514 - accuracy: 0.2982 - val_loss: 3.0822 - val_accuracy: 0.0755
Epoch 5/5
800/800 [==============================] - 23s 28ms/step - loss: 1.8655 - accuracy: 0.3912 - val_loss: 3.4433 - val_accuracy: 0.0759


In [27]:
keras_model.save('PreModel/keras_mbti_classifier.h5')
loaded_svc_model = joblib.load('PreModel/svc_mbti_classifier.pkl')
loaded_vectorizer = joblib.load('PreModel/count_vectorizer.pkl')
loaded_keras_model = tf.keras.models.load_model('PreModel/keras_mbti_classifier.h5')

<h3><i><b>Accuracy and Save Model<b></i></h3>

In [28]:
sample_text = ["enfj since january busy schedule able to spend little time..."]
sample_text_vect = loaded_vectorizer.transform(sample_text)
svc_prediction = loaded_svc_model.predict(sample_text_vect)
print("SVC Prediction:", svc_prediction)

sample_text_array = sample_text_vect.toarray()
keras_prediction = loaded_keras_model.predict(sample_text_array)
predicted_class = df['type'].unique()[np.argmax(keras_prediction)]
print("Keras Prediction:", predicted_class)

SVC Prediction: ['ENFJ']
1/1 [==============================] - 0s 143ms/step
Keras Prediction: ISFJ


<style>
.custom-header {
    background-color: #C70039; 
    color: white; 
    text-align: center; 
    border-radius: 12px;
    font-family: Italic, sans-serif;
    font-size: 40px; 
    box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);
}
</style>
<div class="custom-header">
    Second Chapter: Login/Register Using Face Detection
</div>



In [29]:
import os
import cv2
import customtkinter as ctk
from tkinter import messagebox
from PIL import Image, ImageTk
from deepface import DeepFace
from tensorflow.keras.models import load_model  
from tensorflow.keras.layers import InputLayer
from joblib import load
import numpy as np


<style>
.custom-header {
    background-color: #581845; 
    color: #FF5733; 
    text-align: left; 
    border-radius: 12px;
    padding:10px;
    font-family: Italic, sans-serif;
    font-size: 15px; 
    box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);
}
</style>
<div class="custom-header">
    <h3><i><b>The Main Class That Holds the Necessary Methods</b></i></h3>
    <b>*Constructor</b> - Initializes the pre-trained model and creates the necessary files.<br>
    <br>
    <b>*load_mbti_model</b> - Loads the main model.<br>
    <b>*load_vectorizer</b> - Loads the vectorizer.<br>
    <br>
    <b>*create_main_interface</b> - This is the main menu that includes the login and register buttons.<br>
    <br>
    <b>*register_user_interface</b> - Creates the GUI for the registration page.<br>
    <b>*register_user</b> - When the user clicks on "Register," this method captures the image to save it in the database.<br>
    <br>
    <b>*login_user_interface</b> - Creates the GUI for the login page.<br>
    <b>*user_login</b> - When the user clicks on "Login," this method captures the image, checks it against the database, matches it with the username, and opens the welcome page.<br>
    <br>
     <b>*show_welcome_window</b> - Creates the GUI for the welcome page, where the user can talk about themselves.<br>
    <b>*submit_personality</b> - When the user clicks on "Submit," this method takes the input and passes it to the NLP model to get the personality type.<br>
    <br>
    <b>*deleted_user_window</b> - Creates the GUI that handles the deletion of a user.<br>
    <b>*show_available_users</b> - Searches in the local database for matching usernames.<br><br>
</div>


In [32]:

class Classifier:
    USER_DB_DIR = 'user_db'
    USER_DATA_FILE = 'user_db/users.txt'
    MODEL_PATH = 'PreModel/keras_mbti_classifier.h5'
    users = []
    user_to_delete = []

    def __init__(self, root):
        if not os.path.exists(self.USER_DB_DIR):
            os.makedirs(self.USER_DB_DIR)

        self.root = root
        self.model = self.load_mbti_model()
        self.vectorizer = self.load_vectorizer()
        self.create_main_interface()

    def load_mbti_model(self):
        try:
            model = load_model(self.MODEL_PATH, custom_objects={'InputLayer': InputLayer})
            return model
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load model: {e}")
            return None

    def load_vectorizer(self):
        try:
            vectorizer = load('PreModel/count_vectorizer.pkl')
            return vectorizer
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load vectorizer: {e}")
            return None

    def clear_window(self):
        for widget in self.root.winfo_children():
            widget.destroy()
    def create_main_interface(self):
            self.clear_window()

            image_path = 'images/ai.jpg'
            if os.path.exists(image_path):
                image = Image.open(image_path)
                image = image.resize((200, 200), Image.ADAPTIVE)
                photo = ImageTk.PhotoImage(image)
                image_label = ctk.CTkLabel(self.root, image=photo, text="")
                image_label.image = photo
                image_label.pack(side=ctk.LEFT, padx=(20, 10))

            register_button = ctk.CTkButton(self.root, text="Register", command=self.register_user_interface, width=200, height=50)
            register_button.pack(pady=70, padx=50)

            login_button = ctk.CTkButton(self.root, text="Login", command=self.login_user_interface, width=200, height=50)
            login_button.pack(pady=40, padx=50)


    def register_user_interface(self):
        self.clear_window()

        register_frame = ctk.CTkFrame(self.root)
        register_frame.pack(pady=50, padx=20)

        username_label = ctk.CTkLabel(register_frame, text="Enter your name:")
        username_label.pack(pady=10)

        self.username_entry = ctk.CTkEntry(register_frame, width=200)
        self.username_entry.pack(pady=10)

        register_button = ctk.CTkButton(register_frame, text="Register", command=self.register_user)
        register_button.pack(pady=20)
        
        ctk.CTkButton(self.root, text="Delete User", command=self.deleted_user_window).pack(pady=10)

        back_button = ctk.CTkButton(register_frame, text="Back", command=self.create_main_interface)
        back_button.pack(pady=10)

    def login_user_interface(self):
        self.clear_window()

        login_frame = ctk.CTkFrame(self.root)
        login_frame.pack(pady=20, padx=20)

        username_label = ctk.CTkLabel(login_frame, text="Enter your name:")
        username_label.pack(pady=10)

        self.username_entry = ctk.CTkEntry(login_frame, width=200)
        self.username_entry.pack(pady=10)

        login_button = ctk.CTkButton(login_frame, text="Login", command=self.user_login, width=200)
        login_button.pack(pady=20)
        
        back_button = ctk.CTkButton(login_frame, text="Back", command=self.create_main_interface, width=200)
        back_button.pack(pady=20)

    def register_user(self):
        username = self.username_entry.get().strip()
        if not username:
            messagebox.showerror("Error", "No name entered.")
            return
        self.capture_image(username)

    def capture_image(self, username):
        cap = cv2.VideoCapture(0)
        while True:
            _, frame = cap.read()
            cv2.imshow('S -> Save Q -> Quit', frame)
            key = cv2.waitKey(1)
            if key & 0xFF == ord('s'):
                image_path = os.path.join(self.USER_DB_DIR, f'{username}.jpg')
                cv2.imwrite(image_path, frame)
                messagebox.showinfo("Registration", f"User '{username}' registered successfully!")
                break
            elif key & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

    def user_login(self):
        username = self.username_entry.get().strip()
        if not username:
            messagebox.showerror("Error", "Please enter your username.")
            return

        self.clear_window()
        temp_image_path = os.path.join(self.USER_DB_DIR, 'temp.jpg')
        
        cap = cv2.VideoCapture(0)
        while True:
            _, frame = cap.read()
            cv2.imshow('S -> Save Q -> Quit', frame)
            key = cv2.waitKey(1)
            if key & 0xFF == ord('s'):
                cv2.imwrite(temp_image_path, frame)
                break
            elif key & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                return
        cap.release()
        cv2.destroyAllWindows()

        registered_image_path = os.path.join(self.USER_DB_DIR, f'{username}.jpg')
        if not os.path.exists(registered_image_path):
            messagebox.showerror("Login Failed", "Username not found.")
            os.remove(temp_image_path)
            return
        try:
            result = DeepFace.verify(temp_image_path, registered_image_path, model_name='VGG-Face', enforce_detection=False)
            if result['verified']:
                messagebox.showinfo("Login", f"Welcome, {username}")
                self.show_welcome_window(username)
            else:
                messagebox.showerror("Login Failed", "Face does not match the registered user.")
        except Exception as e:
            messagebox.showerror("Login Failed", f"An error occurred during verification: {e}")
            print(e)
        
        os.remove(temp_image_path)

    def show_welcome_window(self, username):
        self.clear_window()

        welcome_label = ctk.CTkLabel(self.root, text=f"Welcome, {username}\n Write as much as you can about yourself\nthen click 'Submit' to discover your personality type:")
        welcome_label.pack(pady=20)

        input_text = ctk.CTkTextbox(self.root, height=100, width=300)
        input_text.pack(pady=10)

        prediction_label = ctk.CTkLabel(self.root, text="")
        prediction_label.pack(pady=10)

        submit_button = ctk.CTkButton(self.root, text="Submit", command=lambda: self.submit_personality(input_text.get("1.0", "end").strip(), prediction_label), width=200)
        submit_button.pack(pady=20)

        back_button = ctk.CTkButton(self.root, text="Back", command=self.create_main_interface, width=200)
        back_button.pack(pady=20)
    def deleted_user_window(self):
        self.clear_window()
        ctk.CTkLabel(self.root, text="Available users:\n").pack(pady=10)

        scrollable_frame = ctk.CTkScrollableFrame(self.root, width=200, height=120)
        scrollable_frame.pack(pady=10)

        self.users = self.show_avaliable_users()

        def checkbox_event(var, index_to_delete):
            if var.get() == 1:
                self.user_to_delete.append(index_to_delete)
            else:
                self.user_to_delete.remove(index_to_delete)

        for i, user in enumerate(self.users):
            checkbox_var = ctk.IntVar()
            ctk.CTkCheckBox(scrollable_frame, text=f"{user}", variable=checkbox_var,
                            command=lambda var=checkbox_var, idx=i: checkbox_event(var, idx)).pack(pady=5)

        ctk.CTkButton(self.root, text="Delete User", command=lambda: self.delete(self.user_to_delete), width=200).pack(pady=5)
        ctk.CTkButton(self.root, text="Back", command=self.create_main_interface, width=200).pack(pady=20)

    def delete(self,user_to_delete):
        if not user_to_delete:
            messagebox.showerror("Delete User", "There is no user selected. Please try again.")
        else:
            for i in user_to_delete:
                self.deleted_user(self.users[i])

    def deleted_user(self, username):
        user_data = f'{self.USER_DB_DIR}/{username}.jpg'
        if os.path.exists(user_data):
            os.remove(user_data)
            messagebox.showinfo("Delete User", f"The user '{username}' was successfully deleted.")
        else:
            messagebox.showerror("Delete User", "There is no user with this name. Please try again.")
    def show_avaliable_users(self): 
        users=[]
        for x in os.listdir(f"{self.USER_DB_DIR}"):
            if x.endswith(".jpg"):
                users.append(x[:-4])
        return users

    def submit_personality(self, user_input, prediction_label):
        if self.vectorizer is None or self.model is None:
            messagebox.showerror("Error", "Model or vectorizer not initialized.")
            return

        try:
            processed_input = self.vectorizer.transform([user_input])
            prediction = self.model.predict(processed_input.toarray())

            predicted_class_index = np.argmax(prediction, axis=1)[0]

            class_labels = {0: "ISTJ", 1: "ISFJ", 2: "INFJ", 3: "INTJ", 4: "ISTP", 5: "ISFP", 6: "INFP",
                            7: "INTP", 8: "ESTP", 9: "ESFP", 10: "ENFP", 11: "ENTP", 12: "ESTJ", 13: "ESFJ", 14: "ENFJ", 15: "ENTJ"}

            predicted_class_label = class_labels.get(predicted_class_index, "Unknown Class")

            prediction_label.configure(text=f"Predicted Personality: {predicted_class_label}")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred during prediction: {e}")




In [33]:

def main():
    ctk.set_appearance_mode("System")
    ctk.set_default_color_theme("blue")

    root = ctk.CTk()
    root.title("Personality Classifier")
    root.wm_iconbitmap('images/ai.ico')
    Classifier(root)

    root.geometry("500x400")
    root.mainloop()

if __name__ == "__main__":
    main()

1/1 [==============================] - 0s 31ms/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\Programes\anconda\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "d:\Programes\anconda\Lib\site-packages\customtkinter\windows\widgets\ctk_button.py", line 554, in _clicked
    self._command()
  File "C:\Users\ibrah\AppData\Local\Temp\ipykernel_2396\337752341.py", line 195, in <lambda>
    ctk.CTkButton(self.root, text="Delete User", command=lambda: self.delete(self.user_to_delete), width=200).pack(pady=5)
                                                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ibrah\AppData\Local\Temp\ipykernel_2396\337752341.py", line 203, in delete
    self.deleted_user(self.users[i])
                      ~~~~~~~~~~^^^
IndexError: list index out of range
